In [92]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [93]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [94]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [95]:
df = pd.read_csv('transactions_train.csv')
df = df.head()

In [111]:
df.shape

(5, 12)

In [97]:
df_fraud = df[df['is_fraud'] ==1]
df_valid = df[df['is_fraud'] ==0]

In [98]:
# # Create a 4x4 grid of images
# fig, axes = plt.subplots(4, 4, figsize=(10, 10))
# fig.suptitle("First 16 Images", fontsize=16)

# # Plot the first 16 images from the dataset
# for i, ax in enumerate(axes.flat):
#     img = df.iloc[i, 1:].values.reshape(28, 28)  # Reshape to 28x28
#     ax.imshow(img)  # Display in grayscale
#     ax.axis('off')  # Remove axis for a cleaner look
#     ax.set_title(f"Label: {df.iloc[i, 0]}")  # Show the label

# plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to fit the title
# plt.show()


In [99]:
X = df.drop(columns = ['is_fraud'])
y = df['is_fraud']

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [101]:
print(X_train.dtypes)

transaction_amount                    float64
transaction_date                       object
transaction_channel                    object
transaction_payment_mode_anonymous      int64
payment_gateway_bank_anonymous          int64
payer_browser_anonymous                 int64
payer_email_anonymous                  object
payee_ip_anonymous                     object
payer_mobile_anonymous                 object
transaction_id_anonymous               object
payee_id_anonymous                     object
dtype: object


In [102]:
#import onehotencoder
from sklearn.preprocessing import OneHotEncoder

In [103]:
class CustomDataset(Dataset):
    def __init__(self, csv_path):
        df = pd.read_csv(csv_path)

        # Handle missing values
        df['transaction_amount'].fillna(df['transaction_amount'].median(), inplace=True)
        df['payer_mobile_anonymous'].fillna("unknown", inplace=True)

        # Define target and features
        self.labels = torch.tensor(df['is_fraud'].values, dtype=torch.long)

        # Features to use
        numerical_cols = ['transaction_amount', 'transaction_payment_mode_anonymous',
                          'payment_gateway_bank_anonymous', 'payer_browser_anonymous']
        categorical_cols = ['transaction_channel']

        # One-hot encode categorical columns
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded_data = encoder.fit_transform(df[categorical_cols])

        encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

        # Combine numerical and encoded features
        features = pd.concat([df[numerical_cols], encoded_df], axis=1)
        self.features = torch.tensor(features.values, dtype=torch.float32)

        # Save the input dimension for model initialization
        self.input_dim = self.features.shape[1]

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]


In [104]:
train_dataset = CustomDataset("/content/transactions_train.csv")

<ipython-input-103-3ae4cefb0fd4>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['transaction_amount'].fillna(df['transaction_amount'].median(), inplace=True)
<ipython-input-103-3ae4cefb0fd4>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].meth

In [105]:
test_dataset = CustomDataset("/content/transactions_train.csv")

<ipython-input-103-3ae4cefb0fd4>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['transaction_amount'].fillna(df['transaction_amount'].median(), inplace=True)
<ipython-input-103-3ae4cefb0fd4>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].meth

In [106]:
class MyNN(nn.Module):
    def __init__(self, input_dim, output_dim, no_of_HiddenLayer, no_of_neurons, no_of_Dropout):
        super().__init__()
        layers = []

        # First hidden layer
        layers.append(nn.Linear(input_dim, no_of_neurons))
        layers.append(nn.Dropout(no_of_Dropout))
        layers.append(nn.ReLU())
        layers.append(nn.BatchNorm1d(no_of_neurons))

        # Subsequent hidden layers (if any)
        for _ in range(no_of_HiddenLayer - 1):  # Adjust loop to account for first layer
            layers.append(nn.Linear(no_of_neurons, no_of_neurons))  # Input dim is now no_of_neurons
            layers.append(nn.Dropout(no_of_Dropout))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(no_of_neurons))

        # Output layer
        layers.append(nn.Linear(no_of_neurons, output_dim))  # Input dim is no_of_neurons

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [107]:
def optimize(trail):

  # Search Space
   no_of_HiddenLayer = trail.suggest_int('no_of_HiddenLayer', 1 ,5)
   no_of_neurons = trail.suggest_int('no_of_neurons',8,128,step=8)
   no_of_epochs = trail.suggest_int('no_of_epochs',10,100)
   no_of_Dropout = trail.suggest_float('no_of_Dropout',0.1,0.5,step = 0.1)
   no_of_batch_size = trail.suggest_int('no_of_batch_size',16,256,step=16)
   learning_rate = trail.suggest_float('learning_rate',1e-5,1e-1,log=True)
   optimizerFn = trail.suggest_categorical('optimizerFn',['SGD','Adam','RMSprop'])
   weight_decay = trail.suggest_float('weight_decay',1e-5,1e-1,log=True)

   #data loader

   train_loader = DataLoader(train_dataset, batch_size=no_of_batch_size, shuffle=True, pin_memory=True)
   test_loader = DataLoader(test_dataset, batch_size=no_of_batch_size, shuffle=False, pin_memory=True)
   #model init
   model = MyNN(train_dataset.input_dim, 2, no_of_HiddenLayer, no_of_neurons, no_of_Dropout) # Changed output_dim to 2, using train_dataset.input_dim
   model = model.to(device)

  #parameter init
   learning_rate = 0.1
   epochs = 100

   #training Loop


   # loss function
   criterion = nn.CrossEntropyLoss()
  # optimizer
   if optimizerFn == 'Adam':
      optimizer = optim.Adam(model.parameters(), lr=learning_rate,weight_decay=weight_decay)
   elif optimizerFn == 'SGD':
      optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
   elif optimizerFn == 'RMSprop':
      optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
   for epoch in range(epochs):

    total_epoch_loss = 0

    for batch_features, batch_labels in train_loader:

    # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
      outputs = model(batch_features)

    # calculate loss
      loss = criterion(outputs, batch_labels)

    # back pass
      optimizer.zero_grad()
      loss.backward()

    # update grads
      optimizer.step()

    # evalution
    model.eval()
    # evaluation code
    total = 0
    correct = 0

    with torch.no_grad():

      for batch_features, batch_labels in test_loader:

        # move data to gpu
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

        outputs = model(batch_features)

        _, predicted = torch.max(outputs, 1)

        total = total + batch_labels.shape[0]

        correct = correct + (predicted == batch_labels).sum().item()

      accuracy = (correct/total)


    return accuracy

In [108]:
!pip install optuna

In [109]:
import optuna
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(optimize, n_trials=50)


[I 2025-03-21 11:31:19,881] A new study created in memory with name: no-name-af5e4e21-2e00-4823-ab56-e4375ed31f51
[I 2025-03-21 11:31:27,105] Trial 0 finished with value: 0.9999363889756312 and parameters: {'no_of_HiddenLayer': 5, 'no_of_neurons': 112, 'no_of_epochs': 65, 'no_of_Dropout': 0.5, 'no_of_batch_size': 160, 'learning_rate': 0.060643739156257814, 'optimizerFn': 'SGD', 'weight_decay': 0.00013728686848995877}. Best is trial 0 with value: 0.9999363889756312.
[I 2025-03-21 11:31:52,256] Trial 1 finished with value: 0.9999363889756312 and parameters: {'no_of_HiddenLayer': 1, 'no_of_neurons': 64, 'no_of_epochs': 46, 'no_of_Dropout': 0.4, 'no_of_batch_size': 16, 'learning_rate': 0.00012058749368839079, 'optimizerFn': 'SGD', 'weight_decay': 0.007581079964288224}. Best is trial 0 with value: 0.9999363889756312.
[I 2025-03-21 11:31:56,452] Trial 2 finished with value: 0.9999190405144397 and parameters: {'no_of_HiddenLayer': 1, 'no_of_neurons': 48, 'no_of_epochs': 69, 'no_of_Dropout': 0

In [110]:
print(study.best_params)
print(study.best_value)

{'no_of_HiddenLayer': 5, 'no_of_neurons': 112, 'no_of_epochs': 65, 'no_of_Dropout': 0.5, 'no_of_batch_size': 160, 'learning_rate': 0.060643739156257814, 'optimizerFn': 'SGD', 'weight_decay': 0.00013728686848995877}
0.9999363889756312
